<a href="https://colab.research.google.com/github/phongnd205/PointCNN_Face_Part_Segmentation/blob/main/PointCNN_PartSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!git clone https://github.com/yangyanli/PointCNN.git '/content/drive/MyDrive/Colab_Notebooks/PointCNN'

In [ ]:
%tensorflow_version 1.0
import tensorflow as tf

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


In [ ]:
ls /tensorflow-1.15.2/python3.7/tensorflow_core

In [ ]:
cd '/tensorflow-1.15.2/python3.7/tensorflow_core'

In [ ]:
!sudo ln -s libtensorflow_framework.so.1 libtensorflow_framework.so

In [ ]:
cp /tensorflow-1.15.2/python3.7/tensorflow_core/libtensorflow_framework.so /usr/lib/

In [ ]:
cp /tensorflow-1.15.2/python3.7/tensorflow_core/libtensorflow_framework.so.1 /usr/lib/

In [ ]:
cd '/content/drive/MyDrive/Colab_Notebooks/PointCNN/sampling'

In [ ]:
!bash tf_sampling_compile.sh

In [ ]:
cd '/content/drive/MyDrive/Colab_Notebooks/PointCNN'

/content/drive/.shortcut-targets-by-id/1cHCgIPih8muLnTVO67gyq4Ml30OLbACX/PointCNN


In [ ]:
pip install plyfile transforms3d tqdm svgpathtools

In [ ]:
#!/usr/bin/python3
"""Training and Validation On Segmentation Task."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import random
import shutil
import argparse
import importlib
import data_utils
import numpy as np
import pointfly as pf
import tensorflow as tf
from datetime import datetime

In [ ]:
parser = argparse.ArgumentParser()
#parser.add_argument('--filelist', '-t', help='Path to training set ground truth (.txt)', required=True)
#parser.add_argument('--filelist_val', '-v', help='Path to validation set ground truth (.txt)', required=True)
parser.add_argument('--load_ckpt', '-l', help='Path to a check point file for load')
parser.add_argument('--save_folder', default='log2/Segmentation', help='Path to folder for saving check points and summary')
parser.add_argument('--model', default='pointcnn_seg2', help='Model to use')
parser.add_argument('--setting', help='Setting to use')
parser.add_argument('--epochs', type=int, default=2048, help='Number of training epochs (default defined in setting)')
parser.add_argument('--batch_size', type=int, default=8, help='Batch size (default defined in setting)')
parser.add_argument('--learning_rate', type=float, default=0.001, help='Initial learning rate [default: 0.001]')
parser.add_argument('--log', help='Log to FILE in save folder; use - for stdout (default is log.txt)', metavar='FILE', default='log.txt')
parser.add_argument('--no_timestamp_folder', help='Dont save to timestamp folder', action='store_true')
parser.add_argument('--no_code_backup', help='Dont backup code', action='store_true')


In [ ]:
args = parser.parse_args(args=[])

In [ ]:
class setting:
    pass

Setting

In [ ]:
setting.num_class = 50

setting.sample_num = 2048

setting.batch_size = 8

setting.num_epochs = 401

setting.label_weights = []

setting.label_weights = [1.0] * setting.num_class

setting.learning_rate_base = 0.001
setting.decay_steps = 100000
setting.decay_rate = 0.35
setting.learning_rate_min = 0.00001
setting.step_val = 2000

setting.weight_decay = 0.0

setting.jitter = 0.001
setting.jitter_val = 0.0

setting.rotation_range = [0, 0, 0, 'u']
setting.rotation_range_val = [0, 0, 0, 'u']
setting.rotation_order = 'rxyz'

setting.scaling_range = [0.0, 0.0, 0.0, 'g']
setting.scaling_range_val = [0, 0, 0, 'u']

setting.sample_num_variance = 1 // 8
setting.sample_num_clip = 1 // 4

setting.x = 8
x = setting.x
setting.xconv_param_name = ('K', 'D', 'P', 'C', 'links')
setting.xconv_params = [dict(zip(setting.xconv_param_name, xconv_param)) for xconv_param in
                [(8, 1, -1, 32 * x, []),
                 (12, 2, 768, 32 * x, []),
                 (16, 2, 384, 64 * x, []),
                 (16, 6, 128, 128 * x, [])]]

setting.with_global = True

setting.xdconv_param_name = ('K', 'D', 'pts_layer_idx', 'qrs_layer_idx')
setting.xdconv_params = [dict(zip(setting.xdconv_param_name, xdconv_param)) for xdconv_param in
                 [(16, 6, 3, 3),
                  (16, 6, 3, 2),
                  (12, 6, 2, 1),
                  (8, 6, 1, 0),
                  (8, 4, 0, 0)]]

setting.fc_param_name = ('C', 'dropout_rate')
setting.fc_params = [dict(zip(setting.fc_param_name, fc_param)) for fc_param in
             [(32 * x, 0.0),
              (32 * x, 0.5)]]

setting.sampling = 'fps'

setting.optimizer = 'adam'
setting.epsilon = 1e-3
setting.momentum = 0.9

setting.data_dim = 3
setting.with_X_transformation = True
setting.sorting_method = None

setting.keep_remainder = True

In [ ]:
model = importlib.import_module(args.model)

In [ ]:
label_weights_list = setting.label_weights
num_epochs = setting.num_epochs
batch_size = setting.batch_size
step_val = setting.step_val
sample_num = setting.sample_num
rotation_range = setting.rotation_range,
scaling_range = setting.scaling_range,

jitter = setting.jitter
jitter_val = setting.jitter_val

In [ ]:
rotation_range_val = setting.rotation_range_val
scaling_range_val = setting.scaling_range_val
scaling_range = setting.scaling_range

In [ ]:
import numpy as np

In [ ]:
point_data = np.load('/content/drive/MyDrive/Colab_Notebooks/PointCNN/PartSegmentationData_data.npy')
point_label = np.load('/content/drive/MyDrive/Colab_Notebooks/PointCNN/PartSegmentationData_label.npy')

In [ ]:
index = np.arange(0,666)
index = index*4
point_segment_data_label = {"data": point_data[index,:,:], "label": point_label[index,:]}

In [ ]:
data_num = np.full((2664), 4096)

In [ ]:
test_index = np.arange(666*2,666*3)
for_train_index = np.array([i for i in range(len(point_data)) if i not in test_index])
#np.random.shuffle(for_train_index)
train_index = for_train_index[0:1920]
valid_index = for_train_index[1920:1998]

In [ ]:
666*4

2664

Training data

In [ ]:
data_train = point_data[train_index,0:4096,:]
data_num_train = data_num[train_index,]
label_train = point_label[train_index,0:4096]

Validate data

In [ ]:
data_val = point_data[valid_index,0:4096,:]
data_num_val = data_num[valid_index,]
label_val = point_label[valid_index,0:4096]

In [ ]:
del point_data
del point_label

In [ ]:
# shuffle
data_train, data_num_train, label_train = \
    data_utils.grouped_shuffle([data_train, data_num_train, label_train])


In [ ]:
num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]


In [ ]:
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
batch_num_val = math.ceil(num_val / batch_size)

In [ ]:
######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
global_step = tf.Variable(0, trainable=False, name='global_step')
is_training = tf.placeholder(tf.bool, name='is_training')

pts_fts = tf.placeholder(tf.float32, shape=(None, point_num, setting.data_dim), name='pts_fts')
labels_seg = tf.placeholder(tf.int64, shape=(None, point_num), name='labels_seg')
labels_weights = tf.placeholder(tf.float32, shape=(None, point_num), name='labels_weights')


In [ ]:
######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
features_augmented = None
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if setting.use_extra_features:
        if setting.with_normal_feature:
            if setting.data_dim < 6:
                print('Only 3D normals are supported!')
                exit()
            elif setting.data_dim == 6:
                features_augmented = pf.augment(features_sampled, rotations)
            else:
                normals, rest = tf.split(features_sampled, [3, setting.data_dim - 6])
                normals_augmented = pf.augment(normals, rotations)
                features_augmented = tf.concat([normals_augmented, rest], axis=-1)
        else:
            features_augmented = features_sampled
else:
    points_sampled = pts_fts_sampled
points_augmented = pf.augment(points_sampled, xforms, jitter_range)


In [ ]:
labels_sampled = tf.gather_nd(labels_seg, indices=indices, name='labels_sampled')
labels_weights_sampled = tf.gather_nd(labels_weights, indices=indices, name='labels_weight_sampled')

net = model.Net(points_augmented, features_augmented, is_training, setting)
logits = net.logits
probs = tf.nn.softmax(logits, name='probs')
predictions = tf.argmax(probs, axis=-1, name='predictions')

loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels_sampled, logits=logits,
                                                 weights=labels_weights_sampled)

with tf.name_scope('metrics'):
    loss_mean_op, loss_mean_update_op = tf.metrics.mean(loss_op)
    t_1_acc_op, t_1_acc_update_op = tf.metrics.accuracy(labels_sampled, predictions, weights=labels_weights_sampled)
    t_1_per_class_acc_op, t_1_per_class_acc_update_op = \
        tf.metrics.mean_per_class_accuracy(labels_sampled, predictions, setting.num_class,
                                           weights=labels_weights_sampled)
reset_metrics_op = tf.variables_initializer([var for var in tf.local_variables()
                                             if var.name.split('/')[0] == 'metrics'])


In [ ]:
_ = tf.summary.scalar('loss/train', tensor=loss_mean_op, collections=['train'])
_ = tf.summary.scalar('t_1_acc/train', tensor=t_1_acc_op, collections=['train'])
_ = tf.summary.scalar('t_1_per_class_acc/train', tensor=t_1_per_class_acc_op, collections=['train'])

_ = tf.summary.scalar('loss/val', tensor=loss_mean_op, collections=['val'])
_ = tf.summary.scalar('t_1_acc/val', tensor=t_1_acc_op, collections=['val'])
_ = tf.summary.scalar('t_1_per_class_acc/val', tensor=t_1_per_class_acc_op, collections=['val'])

lr_exp_op = tf.train.exponential_decay(setting.learning_rate_base, global_step, setting.decay_steps,
                                       setting.decay_rate, staircase=True)
lr_clip_op = tf.maximum(lr_exp_op, setting.learning_rate_min)
_ = tf.summary.scalar('learning_rate', tensor=lr_clip_op, collections=['train'])
reg_loss = setting.weight_decay * tf.losses.get_regularization_loss()
if setting.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=lr_clip_op, epsilon=setting.epsilon)
elif setting.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr_clip_op, momentum=setting.momentum, use_nesterov=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_op + reg_loss, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

saver = tf.train.Saver(max_to_keep=None)


In [ ]:
root_folder = '/content/drive/MyDrive/Colab_Notebooks/PointCNN/Train/npoint_4096_batch_8_epoch_401_learn_001'

In [ ]:
folder_ckpt = os.path.join(root_folder, 'ckpts_fold_3')
if not os.path.exists(folder_ckpt):
    os.makedirs(folder_ckpt)

folder_summary = os.path.join(root_folder, 'summary')
if not os.path.exists(folder_summary):
    os.makedirs(folder_summary)

In [ ]:
parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))

with tf.Session() as sess:
    summaries_op = tf.summary.merge_all('train')
    summaries_val_op = tf.summary.merge_all('val')
    summary_writer = tf.summary.FileWriter(folder_summary, sess.graph)

    sess.run(init_op)

    # Load the model
    if args.load_ckpt is not None:
        saver.restore(sess, args.load_ckpt)
        print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args.load_ckpt))
    else:
        latest_ckpt = tf.train.latest_checkpoint(folder_ckpt)
        #latest_ckpt = '/content/drive/MyDrive/Colab_Notebooks/PointCNN/Train/npoint_2048_batch_16_epoch_401_learn_01_decay_100000_decaterate_035/ckpts_fold_2/iter-14500'

        if latest_ckpt:
            print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
            saver.restore(sess, latest_ckpt)
            print('{}-Checkpoint loaded from {} (Iter {})'.format(
                datetime.now(), latest_ckpt, sess.run(global_step)))


2021-11-22 12:25:49.619765-Parameter number: 8326770.


In [ ]:


import math
import random
import numpy as np
import tensorflow as tf
from transforms3d.euler import euler2mat


# the returned indices will be used by tf.gather_nd
def get_indices(batch_size, sample_num, point_num, pool_setting=None):
    if not isinstance(point_num, np.ndarray):
        point_nums = np.full((batch_size), point_num)
    else:
        point_nums = point_num

    indices = []
    for i in range(batch_size):
        pt_num = point_nums[i]
        if pool_setting is None:
            pool_size = pt_num
        else:
            if isinstance(pool_setting, int):
                pool_size = min(pool_setting, pt_num)
            elif isinstance(pool_setting, tuple):
                pool_size = min(random.randrange(pool_setting[0], pool_setting[1]+1), pt_num)
        if pool_size > sample_num:
            choices = np.random.choice(pool_size, sample_num, replace=False)
        else:
            choices = np.concatenate((np.random.choice(pool_size, pool_size, replace=False),
                                      np.random.choice(pool_size, sample_num - pool_size, replace=True)))
        if pool_size < pt_num:
            choices_pool = np.random.choice(pt_num, pool_size, replace=False)
            choices = choices_pool[choices]
        choices = np.expand_dims(choices, axis=1)
        choices_2d = np.concatenate((np.full_like(choices, i), choices), axis=1)
        indices.append(choices_2d)
    return np.stack(indices)


def gauss_clip(mu, sigma, clip):
    v = random.gauss(mu, sigma)
    v = max(min(v, mu + clip * sigma), mu - clip * sigma)
    return v


def uniform(bound):
    return bound * (2 * random.random() - 1)


def scaling_factor(scaling_param, method):
    try:
        scaling_list = list(scaling_param)
        return random.choice(scaling_list)
    except:
        if method == 'g':
            return gauss_clip(1.0, scaling_param, 3)
        elif method == 'u':
            return 1.0 + uniform(scaling_param)


def rotation_angle(rotation_param, method):
    try:
        rotation_list = list(rotation_param)
        return random.choice(rotation_list)
    except:
        if method == 'g':
            return gauss_clip(0.0, rotation_param, 3)
        elif method == 'u':
            return uniform(rotation_param)


def get_xforms(xform_num, rotation_range, scaling_range, order='rxyz'):
    xforms = np.empty(shape=(xform_num, 3, 3))
    rotations = np.empty(shape=(xform_num, 3, 3))
    for i in range(xform_num):
        rx = rotation_angle(rotation_range[0][0], rotation_range[0][3])
        ry = rotation_angle(rotation_range[0][1], rotation_range[0][3])
        rz = rotation_angle(rotation_range[0][2], rotation_range[0][3])
        rotation = euler2mat(rx, ry, rz, order)

        sx = scaling_factor(scaling_range[0][0], scaling_range[0][3])
        sy = scaling_factor(scaling_range[0][1], scaling_range[0][3])
        sz = scaling_factor(scaling_range[0][2], scaling_range[0][3])
        scaling = np.diag([sx, sy, sz])

        xforms[i, :] = scaling * rotation
        rotations[i, :] = rotation
    return xforms, rotations


def augment(points, xforms, range=None):
    points_xformed = tf.matmul(points, xforms, name='points_xformed')
    if range is None:
        return points_xformed

    jitter_data = range * tf.random_normal(tf.shape(points_xformed), name='jitter_data')
    jitter_clipped = tf.clip_by_value(jitter_data, -5 * range, 5 * range, name='jitter_clipped')
    return points_xformed + jitter_clipped


# A shape is (N, C)
def distance_matrix(A):
    r = tf.reduce_sum(A * A, 1, keep_dims=True)
    m = tf.matmul(A, tf.transpose(A))
    D = r - 2 * m + tf.transpose(r)
    return D


# A shape is (N, P, C)
def batch_distance_matrix(A):
    r = tf.reduce_sum(A * A, axis=2, keep_dims=True)
    m = tf.matmul(A, tf.transpose(A, perm=(0, 2, 1)))
    D = r - 2 * m + tf.transpose(r, perm=(0, 2, 1))
    return D


# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)
def batch_distance_matrix_general(A, B):
    r_A = tf.reduce_sum(A * A, axis=2, keep_dims=True)
    r_B = tf.reduce_sum(B * B, axis=2, keep_dims=True)
    m = tf.matmul(A, tf.transpose(B, perm=(0, 2, 1)))
    D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 2, 1))
    return D


# A shape is (N, P, C)
def find_duplicate_columns(A):
    N = A.shape[0]
    P = A.shape[1]
    indices_duplicated = np.fill((N, 1, P), 1, dtype=np.int32)
    for idx in range(N):
        _, indices = np.unique(A[idx], return_index=True, axis=0)
        indices_duplicated[idx, :, indices] = 0
    return indices_duplicated


# add a big value to duplicate columns
def prepare_for_unique_top_k(D, A):
    indices_duplicated = tf.py_func(find_duplicate_columns, [A], tf.int32)
    D += tf.reduce_max(D)*tf.cast(indices_duplicated, tf.float32)


# return shape is (N, P, K, 2)
def knn_indices(points, k, sort=True, unique=True):
    points_shape = tf.shape(points)
    batch_size = points_shape[0]
    point_num = points_shape[1]

    D = batch_distance_matrix(points)
    if unique:
        prepare_for_unique_top_k(D, points)
    distances, point_indices = tf.nn.top_k(-D, k=k, sorted=sort)
    batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1, 1)), (1, point_num, k, 1))
    indices = tf.concat([batch_indices, tf.expand_dims(point_indices, axis=3)], axis=3)
    return -distances, indices


# return shape is (N, P, K, 2)
def knn_indices_general(queries, points, k, sort=True, unique=True):
    queries_shape = tf.shape(queries)
    batch_size = queries_shape[0]
    point_num = queries_shape[1]

    D = batch_distance_matrix_general(queries, points)
    if unique:
        prepare_for_unique_top_k(D, points)
    distances, point_indices = tf.nn.top_k(-D, k=k, sorted=sort)  # (N, P, K)
    batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1, 1)), (1, point_num, k, 1))
    indices = tf.concat([batch_indices, tf.expand_dims(point_indices, axis=3)], axis=3)
    return -distances, indices


# indices is (N, P, K, 2)
# return shape is (N, P, K, 2)
def sort_points(points, indices, sorting_method):
    indices_shape = tf.shape(indices)
    batch_size = indices_shape[0]
    point_num = indices_shape[1]
    k = indices_shape[2]

    nn_pts = tf.gather_nd(points, indices)  # (N, P, K, 3)
    if sorting_method.startswith('c'):
        if ''.join(sorted(sorting_method[1:])) != 'xyz':
            print('Unknown sorting method!')
            exit()
        epsilon = 1e-8
        nn_pts_min = tf.reduce_min(nn_pts, axis=2, keep_dims=True)
        nn_pts_max = tf.reduce_max(nn_pts, axis=2, keep_dims=True)
        nn_pts_normalized = (nn_pts - nn_pts_min) / (nn_pts_max - nn_pts_min + epsilon)  # (N, P, K, 3)
        scaling_factors = [math.pow(100.0, 3 - sorting_method.find('x')),
                           math.pow(100.0, 3 - sorting_method.find('y')),
                           math.pow(100.0, 3 - sorting_method.find('z'))]
        scaling = tf.constant(scaling_factors, shape=(1, 1, 1, 3))
        sorting_data = tf.reduce_sum(nn_pts_normalized * scaling, axis=-1)  # (N, P, K)
        sorting_data = tf.concat([tf.zeros((batch_size, point_num, 1)), sorting_data[:, :, 1:]], axis=-1)
    elif sorting_method == 'l2':
        nn_pts_center = tf.reduce_mean(nn_pts, axis=2, keep_dims=True)  # (N, P, 1, 3)
        nn_pts_local = tf.subtract(nn_pts, nn_pts_center)  # (N, P, K, 3)
        sorting_data = tf.norm(nn_pts_local, axis=-1)  # (N, P, K)
    else:
        print('Unknown sorting method!')
        exit()
    _, k_indices = tf.nn.top_k(sorting_data, k=k, sorted=True)  # (N, P, K)
    batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1, 1)), (1, point_num, k, 1))
    point_indices = tf.tile(tf.reshape(tf.range(point_num), (1, -1, 1, 1)), (batch_size, 1, k, 1))
    k_indices_4d = tf.expand_dims(k_indices, axis=3)
    sorting_indices = tf.concat([batch_indices, point_indices, k_indices_4d], axis=3)  # (N, P, K, 3)
    return tf.gather_nd(indices, sorting_indices)


# a b c
# d e f
# g h i
# a(ei − fh) − b(di − fg) + c(dh − eg)
def compute_determinant(A):
    return A[..., 0, 0] * (A[..., 1, 1] * A[..., 2, 2] - A[..., 1, 2] * A[..., 2, 1]) \
           - A[..., 0, 1] * (A[..., 1, 0] * A[..., 2, 2] - A[..., 1, 2] * A[..., 2, 0]) \
           + A[..., 0, 2] * (A[..., 1, 0] * A[..., 2, 1] - A[..., 1, 1] * A[..., 2, 0])


# A shape is (N, P, 3, 3)
# return shape is (N, P, 3)
def compute_eigenvals(A):
    A_11 = A[:, :, 0, 0]  # (N, P)
    A_12 = A[:, :, 0, 1]
    A_13 = A[:, :, 0, 2]
    A_22 = A[:, :, 1, 1]
    A_23 = A[:, :, 1, 2]
    A_33 = A[:, :, 2, 2]
    I = tf.eye(3)
    p1 = tf.square(A_12) + tf.square(A_13) + tf.square(A_23)  # (N, P)
    q = tf.trace(A) / 3  # (N, P)
    p2 = tf.square(A_11 - q) + tf.square(A_22 - q) + tf.square(A_33 - q) + 2 * p1  # (N, P)
    p = tf.sqrt(p2 / 6) + 1e-8  # (N, P)
    N = tf.shape(A)[0]
    q_4d = tf.reshape(q, (N, -1, 1, 1))  # (N, P, 1, 1)
    p_4d = tf.reshape(p, (N, -1, 1, 1))
    B = (1 / p_4d) * (A - q_4d * I)  # (N, P, 3, 3)
    r = tf.clip_by_value(compute_determinant(B) / 2, -1, 1)  # (N, P)
    phi = tf.acos(r) / 3  # (N, P)
    eig1 = q + 2 * p * tf.cos(phi)  # (N, P)
    eig3 = q + 2 * p * tf.cos(phi + (2 * math.pi / 3))
    eig2 = 3 * q - eig1 - eig3
    return tf.abs(tf.stack([eig1, eig2, eig3], axis=2))  # (N, P, 3)


# P shape is (N, P, 3), N shape is (N, P, K, 3)
# return shape is (N, P)
def compute_curvature(nn_pts):
    nn_pts_mean = tf.reduce_mean(nn_pts, axis=2, keep_dims=True)  # (N, P, 1, 3)
    nn_pts_demean = nn_pts - nn_pts_mean  # (N, P, K, 3)
    nn_pts_NPK31 = tf.expand_dims(nn_pts_demean, axis=-1)
    covariance_matrix = tf.matmul(nn_pts_NPK31, nn_pts_NPK31, transpose_b=True)  # (N, P, K, 3, 3)
    covariance_matrix_mean = tf.reduce_mean(covariance_matrix, axis=2)  # (N, P, 3, 3)
    eigvals = compute_eigenvals(covariance_matrix_mean)  # (N, P, 3)
    curvature = tf.reduce_min(eigvals, axis=-1) / (tf.reduce_sum(eigvals, axis=-1) + 1e-8)
    return curvature


def curvature_based_sample(nn_pts, k):
    curvature = compute_curvature(nn_pts)
    _, point_indices = tf.nn.top_k(curvature, k=k, sorted=False)

    pts_shape = tf.shape(nn_pts)
    batch_size = pts_shape[0]
    batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1)), (1, k, 1))
    indices = tf.concat([batch_indices, tf.expand_dims(point_indices, axis=2)], axis=2)
    return indices


def random_choice_2d(size, prob_matrix):
    n_row = prob_matrix.shape[0]
    n_col = prob_matrix.shape[1]
    choices = np.ones((n_row, size), dtype=np.int32)
    for idx_row in range(n_row):
        choices[idx_row] = np.random.choice(n_col, size=size, replace=False, p=prob_matrix[idx_row])
    return choices


def inverse_density_sampling(points, k, sample_num):
    D = batch_distance_matrix(points)
    distances, _ = tf.nn.top_k(-D, k=k, sorted=False)
    distances_avg = tf.abs(tf.reduce_mean(distances, axis=-1)) + 1e-8
    prob_matrix = distances_avg / tf.reduce_sum(distances_avg, axis=-1, keep_dims=True)
    point_indices = tf.py_func(random_choice_2d, [sample_num, prob_matrix], tf.int32)
    point_indices.set_shape([points.get_shape()[0], sample_num])

    batch_size = tf.shape(points)[0]
    batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1)), (1, sample_num, 1))
    indices = tf.concat([batch_indices, tf.expand_dims(point_indices, axis=2)], axis=2)
    return indices


def batch_normalization(data, is_training, name, reuse=None):
    return tf.layers.batch_normalization(data, momentum=0.99, training=is_training,
                                         beta_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                                         gamma_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                                         reuse=reuse, name=name)


def separable_conv2d(input, output, name, is_training, kernel_size, depth_multiplier=1,
                     reuse=None, with_bn=True, activation=tf.nn.elu):
    conv2d = tf.layers.separable_conv2d(input, output, kernel_size=kernel_size, strides=(1, 1), padding='VALID',
                                        activation=activation,
                                        depth_multiplier=depth_multiplier,
                                        depthwise_initializer=tf.glorot_normal_initializer(),
                                        pointwise_initializer=tf.glorot_normal_initializer(),
                                        depthwise_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                                        pointwise_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                                        reuse=reuse, name=name, use_bias=not with_bn)
    return batch_normalization(conv2d, is_training, name + '_bn', reuse) if with_bn else conv2d


def depthwise_conv2d(input, depth_multiplier, name, is_training, kernel_size,
                     reuse=None, with_bn=True, activation=tf.nn.elu):
    conv2d = tf.contrib.layers.separable_conv2d(input, num_outputs=None, kernel_size=kernel_size, padding='VALID',
                                                activation_fn=activation,
                                                depth_multiplier=depth_multiplier,
                                                weights_initializer=tf.glorot_normal_initializer(),
                                                weights_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                                                biases_initializer=None if with_bn else tf.zeros_initializer(),
                                                biases_regularizer=None if with_bn else tf.contrib.layers.l2_regularizer(
                                                    scale=1.0),
                                                reuse=reuse, scope=name)
    return batch_normalization(conv2d, is_training, name + '_bn', reuse) if with_bn else conv2d


def conv2d(input, output, name, is_training, kernel_size,
           reuse=None, with_bn=True, activation=tf.nn.elu):
    conv2d = tf.layers.conv2d(input, output, kernel_size=kernel_size, strides=(1, 1), padding='VALID',
                              activation=activation,
                              kernel_initializer=tf.glorot_normal_initializer(),
                              kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                              reuse=reuse, name=name, use_bias=not with_bn)
    return batch_normalization(conv2d, is_training, name + '_bn', reuse) if with_bn else conv2d


def dense(input, output, name, is_training, reuse=None, with_bn=True, activation=tf.nn.elu):
    dense = tf.layers.dense(input, units=output, activation=activation,
                            kernel_initializer=tf.glorot_normal_initializer(),
                            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1.0),
                            reuse=reuse, name=name, use_bias=not with_bn)
    return batch_normalization(dense, is_training, name + '_bn', reuse) if with_bn else dense


In [ ]:
def get_xforms(xform_num, order='rxyz'):
    xforms = np.empty(shape=(xform_num, 3, 3))
    rotations = np.empty(shape=(xform_num, 3, 3))
    for i in range(xform_num):
        rx = rotation_angle(0, 'u')
        ry = rotation_angle(0, 'u')
        rz = rotation_angle(0, 'u')
        rotation = euler2mat(rx, ry, rz, order)

        sx = scaling_factor(0.0, 'g')
        sy = scaling_factor(0.0, 'g')
        sz = scaling_factor(0.0, 'g')
        scaling = np.diag([sx, sy, sz])

        xforms[i, :] = scaling * rotation
        rotations[i, :] = rotation
    return xforms, rotations

In [ ]:
with tf.Session() as sess:
    summaries_op = tf.summary.merge_all('train')
    summaries_val_op = tf.summary.merge_all('val')
    summary_writer = tf.summary.FileWriter(folder_summary, sess.graph)

    sess.run(init_op)

    # Load the model
    if args.load_ckpt is not None:
        saver.restore(sess, args.load_ckpt)
        print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args.load_ckpt))
    else:
        latest_ckpt = tf.train.latest_checkpoint(folder_ckpt)
        #latest_ckpt = '/content/drive/MyDrive/Colab_Notebooks/PointCNN/Train/npoint_2048_batch_16_epoch_401_learn_01_decay_100000_decaterate_035/ckpts_fold_2/iter-14500'
        if latest_ckpt:
            print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
            saver.restore(sess, latest_ckpt)
            print('{}-Checkpoint loaded from {} (Iter {})'.format(
                datetime.now(), latest_ckpt, sess.run(global_step)))

    for batch_idx_train in range(batch_num):
        ######################################################################
        # Validation
        if (batch_idx_train % step_val == 0 and (batch_idx_train != 0 or args.load_ckpt is not None)) \
                or batch_idx_train == batch_num - 1:
            filename_ckpt = os.path.join(folder_ckpt, 'iter')
            saver.save(sess, filename_ckpt, global_step=global_step)
            print('{}-Checkpoint saved to {}!'.format(datetime.now(), filename_ckpt))

            sess.run(reset_metrics_op)
            for batch_val_idx in range(batch_num_val):
                start_idx = batch_size * batch_val_idx
                end_idx = min(start_idx + batch_size, num_val)
                batch_size_val = end_idx - start_idx
                points_batch = data_val[start_idx:end_idx, ...]
                points_num_batch = data_num_val[start_idx:end_idx, ...]
                labels_batch = label_val[start_idx:end_idx, ...].astype(int)
                weights_batch = np.array(label_weights_list)[labels_batch]

                xforms_np, rotations_np = get_xforms(batch_size_val,
                                                        order=setting.rotation_order)
                sess.run([loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                         feed_dict={
                             pts_fts: points_batch,
                             indices: get_indices(batch_size_val, sample_num, points_num_batch),
                             xforms: xforms_np,
                             rotations: rotations_np,
                             jitter_range: np.array([jitter_val]),
                             labels_seg: labels_batch,
                             labels_weights: weights_batch,
                             is_training: False,
                         })
            loss_val, t_1_acc_val, t_1_per_class_acc_val, summaries_val, step = sess.run(
                [loss_mean_op, t_1_acc_op, t_1_per_class_acc_op, summaries_val_op, global_step])
            summary_writer.add_summary(summaries_val, step)
            print('{}-[Val  ]-Average:      Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), loss_val, t_1_acc_val, t_1_per_class_acc_val))
            sys.stdout.flush()
        ######################################################################

        ######################################################################
        # Training
        start_idx = (batch_size * batch_idx_train) % num_train
        end_idx = min(start_idx + batch_size, num_train)
        batch_size_train = end_idx - start_idx
        points_batch = data_train[start_idx:end_idx, ...]
        points_num_batch = data_num_train[start_idx:end_idx, ...]
        labels_batch = label_train[start_idx:end_idx, ...].astype(int)
        weights_batch = np.array(label_weights_list)[labels_batch]

        if start_idx + batch_size_train == num_train:
            data_train, data_num_train, label_train = \
                data_utils.grouped_shuffle([data_train, data_num_train, label_train])

        offset = int(random.gauss(0, sample_num * setting.sample_num_variance))
        offset = max(offset, int(-sample_num * setting.sample_num_clip))
        offset = min(offset, int(sample_num * setting.sample_num_clip))
        sample_num_train = sample_num + offset
        xforms_np, rotations_np = get_xforms(batch_size_train,
                                                order=setting.rotation_order)
        
        sess.run(reset_metrics_op)
        sess.run([train_op, loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                 feed_dict={
                     pts_fts: points_batch,
                     indices: get_indices(batch_size_train, sample_num_train, points_num_batch),
                     xforms: xforms_np,
                     rotations: rotations_np,
                     jitter_range: np.array([jitter]),
                     labels_seg: labels_batch,
                     labels_weights: weights_batch,
                     is_training: True,
                 })
        if batch_idx_train % 10 == 0:
            loss, t_1_acc, t_1_per_class_acc, summaries, step = sess.run([loss_mean_op,
                                                                    t_1_acc_op,
                                                                    t_1_per_class_acc_op,
                                                                    summaries_op,
                                                                    global_step])
            summary_writer.add_summary(summaries, step)
            print('{}-[Train]-Iter: {:06d}  Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), step, loss, t_1_acc, t_1_per_class_acc))
            sys.stdout.flush()
        ######################################################################
    print('{}-Done!'.format(datetime.now()))

In [ ]:
label_weights_list

**Testing Process**

In [ ]:
root_folder = '/content/drive/MyDrive/Colab_Notebooks/PointCNN/predict/npoint_1300'

In [ ]:
folder_predict_fold_1 = os.path.join(root_folder, 'predict_fold_4_Patient')
if not os.path.exists(folder_predict_fold_1):
    os.makedirs(folder_predict_fold_1)

#folder_true_fold_1 = os.path.join(root_folder, 'true_fold_3')
#if not os.path.exists(folder_true_fold_1):
#    os.makedirs(folder_true_fold_1)

In [ ]:
parser1 = argparse.ArgumentParser()
parser1.add_argument('--category', '-c', help='Path to category list file (.txt)')

parser1.add_argument('--load_ckpt', default='/content/drive/MyDrive/Colab_Notebooks/PointCNN/ckpts_fold_1', help='Path to a check point file for load')
parser1.add_argument('--repeat_num', help='Repeat number', type=int, default=1)
parser1.add_argument('--sample_num', help='Point sample num', type=int, default=1300)
parser1.add_argument('--model', default='pointcnn_seg2', help='Model to use')

parser1.add_argument('--save_ply', help='Save results as ply', action='store_true')


_StoreTrueAction(option_strings=['--save_ply'], dest='save_ply', nargs=0, const=True, default=False, type=None, choices=None, help='Save results as ply', metavar=None)

In [ ]:
args1 = parser1.parse_args(args=[])

Train & Test data

In [ ]:
point_data = np.load('/content/drive/MyDrive/Colab_Notebooks/PointCNN/PartSegmentationData_data.npy')
point_label = np.load('/content/drive/MyDrive/Colab_Notebooks/PointCNN/PartSegmentationData_label.npy')

In [ ]:
test_index = np.arange(666*0,666*1)
for_train_index = np.array([i for i in range(len(point_data)) if i not in test_index])
#np.random.shuffle(for_train_index)
train_index = for_train_index[0:1920]
valid_index = for_train_index[1920:1998]

data_num = np.full((2664), 1024)
# Prepare inputs
data = point_data[test_index,0:1024,:]
data_num = data_num[test_index,]
label = point_label[test_index,0:1024]

Patient Data

In [ ]:
point_data = np.load('/content/drive/MyDrive/Colab_Notebooks/PointCNN/PartSegmentationData_PatientData_Norm_2_Kinect1234_MRI1243.npy')
data_num = np.full((16), 1300)
# Prepare inputs
data = point_data[:,0:1300,:]
data_num = data_num[:,]

In [ ]:
model = importlib.import_module(args1.model)

batch_num = data.shape[0]
max_point_num = data.shape[1]

batch_size = args1.repeat_num * math.ceil(data.shape[1] / setting.sample_num)

In [ ]:
print('{}-{:d} testing batches.'.format(datetime.now(), batch_num))

######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(batch_size, None, 2), name="indices")
is_training = tf.placeholder(tf.bool, name='is_training')
pts_fts = tf.placeholder(tf.float32, shape=(None, max_point_num, setting.data_dim), name='pts_fts')
######################################################################

######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if not setting.use_extra_features:
        features_sampled = None
else:
    points_sampled = pts_fts_sampled
    features_sampled = None


2021-11-23 09:06:51.139557-16 testing batches.


In [ ]:
net = model.Net(points_sampled, features_sampled, is_training, setting)
logits = net.logits
probs_op = tf.nn.softmax(logits, name='probs')

saver = tf.train.Saver()

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.co

In [ ]:
sample_num = setting.sample_num

In [ ]:
sample_num = 1300

In [ ]:
import scipy.io

In [ ]:
with tf.Session() as sess:
    # Load the model
    saver.restore(sess, '/content/drive/MyDrive/Colab_Notebooks/PointCNN/Train/npoint_2048_batch_16_epoch_401_learn_01_decay_200000_decaterate_035/ckpts_fold_1/iter-30000')
    print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args1.load_ckpt))

    indices_batch_indices = np.tile(np.reshape(np.arange(batch_size), (batch_size, 1, 1)), (1, sample_num, 1))
    for batch_idx in range(batch_num):
        points_batch = data[[batch_idx] * batch_size, ...]
        #object_label = label[batch_idx]
        point_num = data_num[batch_idx]
        #category = category_list[object_label][0]
        #label_start, label_end = category_range[category]

        tile_num = math.ceil((sample_num * batch_size) / point_num)
        indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
        np.random.shuffle(indices_shuffle)
        indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
        indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)

        probs = sess.run([probs_op],
                            feed_dict={
                                pts_fts: points_batch,
                                indices: indices_batch,
                                is_training: False,
                            })
        probs_2d = np.reshape(probs, (sample_num * batch_size, -1))
        predictions = [(-1, 0.0)] * point_num
        for idx in range(sample_num * batch_size):
            point_idx = indices_shuffle[idx]
            probs = probs_2d[idx, :]
            confidence = np.amax(probs)
            seg_idx = np.argmax(probs)
            if confidence > predictions[point_idx][1]:
                predictions[point_idx] = (seg_idx, confidence)

        
        labels = []
        for seg_idx, _ in predictions:
            labels.append(seg_idx)
        
        arr = labels
        scipy.io.savemat('/content/drive/MyDrive/Colab_Notebooks/PointCNN/predict/npoint_1300/predict_fold_1_Patient/'+ str(batch_idx) + '.mat', mdict={'arr': arr})
        labels = []
        
        #with open('/content/drive/MyDrive/Colab_Notebooks/PointCNN/output_seg_predict/'+ str(batch_idx) + '.txt', 'w') as file_seg:
        #    for seg_idx, _ in predictions:
        #        file_seg.write('%d\n' % (seg_idx))
        #        labels.append(seg_idx)
        #    file_seg.close() 

        sys.stdout.flush()
        ######################################################################
    print('{}-Done!'.format(datetime.now()))

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Colab_Notebooks/PointCNN/Train/npoint_2048_batch_16_epoch_401_learn_01_decay_200000_decaterate_035/ckpts_fold_1/iter-30000
2021-11-23 09:08:01.964057-Checkpoint loaded from /content/drive/MyDrive/Colab_Notebooks/PointCNN/ckpts_fold_1!
2021-11-23 09:08:04.549722-Done!
